In [1]:
import random, time


class Creature:
    def __init__(self, name, HP=10):
        self.name = name
        self.HP = HP
        self.maxHP = HP
        self.abilities = {
            "Attack":1,
            "Defense":5,
            "Speed":5
        }
    
    def set_HP(self, value):
        self.HP = value
        if self.HP > self.maxHP:
            self.HP = self.maxHP
        
    def get_HP(self):
        return self.HP 
        
    def check_life(self):
        if self.HP <= 0:
            self.HP = 0
            print("\033[91m" + f"{self.name} has been dusted." + "\033[0m")
        return self.HP
    
    def attack(self, other):
        print(f"{self.name} attacks {other.name}")
        other_att_spd = other.abilities["Defense"] + other.abilities["Speed"]
        chance = random.randint(1,20)
        if chance > other_att_spd:
            otherHP = other.get_HP()
            damage = self.abilities["Attack"] + random.randint(1,4)
            otherHP -= damage
            other.set_HP(otherHP)
            if damage > 0:
                print(f"Attack hits for {damage} damage!")
        else:
            print("The attack failed")
    
    def auto_select(self, target_list):
        if len(target_list) > 0:
            return random.choice(target_list)
        return None
        
    def turn(self, round_num, target_list):
        target = self.auto_select(target_list)
        if target == None:
            return False
        self.attack(target)
        if target.check_life() == 0:
            target_list.remove(target)
        return True

Created the class creature and in the constructor __init__() added the variables for name, HP, maxHP and abilities. The name is passed from the calling of the class instance, the HP is set to a default value of 10, but can optionally be passed in. MaxHP is set to HP. For abilities I chose to store them in a dictionary. Originally I had them passed in as variables with default values but changed this as nothing requires changing them on initiation. Also, I didn't pass it attack, strength and defense as it would mean creating an instance of the creature class would take longer.

check_life() - is called it will return the value of HP and print a message if it was less than or equal to 0 it prints a message to that affect to indicate the creature is dead.

attack(target) - this prints a message to show who its attacking, runs the formula for attacking and then either calculates the new HP or prints that the attack failed.

auto_select(target_list) - The list passed to auto select always has targets with health, as any with no health are removed. It checks that there is elements in the list then returns either a random element or None.

turn(round_num, target_list) - this calls auto_select() to get a target. It will return False if target is None. It then attacks the target. It calls check_life and if the life is 0 it will print the death message and remove the target from the list.

get_HP() is used to get the HP level instead of using check_life(). This is as it won't print out the deceased message.

set_HP() - was initially just returning HP. With the introduction of healing it checks if it's above maxHP then sets HP to maxHP

In [2]:
Pippin = Creature("Pippin")
Frodo = Creature("Frodo")
Bilbo = Creature("Bilbo")
Sarumon = Creature("Sarumon")

creature_list1 = [Pippin, Frodo]
creature_list2 = [Bilbo, Sarumon]


def testgame(creature_list1, creature_list2):
    """tests the Creatures instances in the list against each other

    Args:
        creature_list1 (list): list of instances from Creature class and children classes
        creature_list2 (list): list of instances from Creature class and children classes
    """
    doTurn = True

    for round in range(1,20):
        if not doTurn:
            break
        print(f"Round: {round}")
        for creature in creature_list1:
            if doTurn:
                doTurn = creature.turn(round, creature_list2)
            
        for creature in creature_list2:
            if doTurn:
                doTurn = creature.turn(round, creature_list1)
                
        if len(creature_list1) == 0 or len(creature_list2) == 0:
            doTurn = False

testgame(creature_list1, creature_list2)

    

Round: 1
Pippin attacks Bilbo
Attack hits for 2 damage!
Frodo attacks Bilbo
Attack hits for 4 damage!
Bilbo attacks Pippin
The attack failed
Sarumon attacks Frodo
The attack failed
Round: 2
Pippin attacks Bilbo
Attack hits for 4 damage!
Bilbo has been dusted.
Frodo attacks Sarumon
Attack hits for 4 damage!
Sarumon attacks Pippin
The attack failed
Round: 3
Pippin attacks Sarumon
The attack failed
Frodo attacks Sarumon
The attack failed
Sarumon attacks Pippin
The attack failed
Round: 4
Pippin attacks Sarumon
The attack failed
Frodo attacks Sarumon
The attack failed
Sarumon attacks Pippin
Attack hits for 2 damage!
Round: 5
Pippin attacks Sarumon
Attack hits for 5 damage!
Frodo attacks Sarumon
The attack failed
Sarumon attacks Frodo
The attack failed
Round: 6
Pippin attacks Sarumon
Attack hits for 5 damage!
Sarumon has been dusted.


The above code writes a formula to test the game, testgame(). This will be used for all future tests. It runs the rounds for 50 rounds using the forloop. It then checks doTurn for each creature in the list. If do turn is still true the creature takes a turn. When doTurn is False it breaks the loop early as all creatures are dead.

In [3]:
class Goblin(Creature):
    def __init__(self, name, HP=15):
        Creature.__init__(self, name, HP)
        self.abilities["Attack"] = 3
        self.abilities["Defense"] = 6
        self.abilities["Speed"] = 6

        
class Orc(Creature):
    def __init__(self, name, HP=50):
        Creature.__init__(self, name, HP)
        self.abilities["Attack"] = 5
        self.abilities["Defense"] = 8
        self.abilities["Speed"] = 3
        self.rage = False
    
    def heavy_attack(self, other):
        if self.rage == False:
            self.rage = True
            print(f"{self.name} is in RAAAAAGE")
            self.abilities["Attack"] += 5
            self.abilities["Defense"] -= 3
        super().attack(other)
    
    def attack(self, other):
        if self.rage == True:
            self.rage = False
            self.abilities["Attack"] -= 5
            self.abilities["Defense"] += 3  
            print(f"{self.name} cooled down")
        super().attack(other)
    
    def turn(self, round_num, target_list):
        target = self.auto_select(target_list)
        if target == None:
            return False
        if round_num % 4 == 0:
            self.heavy_attack(target)
        else: 
            self.attack(target)        
        if target.check_life() == 0:
            target_list.remove(target)
        return True


Goblin - in the __init__() it calls the creature init and then changes the abilities to the required amounts. It passes the HP to the creature HP to set the higher amount.

Orc - This does the same as Goblin. It adds another attribute, self.rage as False. This will be used later. 

heavy_attack() checks if self.rage is False. It prints a message then adds to attack and subtracts from defense. It calls super().attack(other) to use creatures normal attack instead of the new attack() method defined in Orc.

attack() checks the flag variable self.rage then lowers the abilities and prints a message to that affect before calling super().attack.

turn() takes the round number and target_list in the same was as creature. It checks if round_num % 4 is 0 and uses a heavy attack, otherwise uses normal attack. 

In [4]:
Pippin = Goblin("Gobbers1")
Frodo = Orc("Orc1")
Bilbo = Goblin("Gobbers2")
Sarumon = Orc("Orc2")

creature_list1 = [Pippin, Frodo]
creature_list2 = [Bilbo, Sarumon]

testgame(creature_list1, creature_list2)


Round: 1
Gobbers1 attacks Orc2
The attack failed
Orc1 attacks Gobbers2
The attack failed
Gobbers2 attacks Gobbers1
Attack hits for 5 damage!
Orc2 attacks Orc1
Attack hits for 7 damage!
Round: 2
Gobbers1 attacks Orc2
Attack hits for 4 damage!
Orc1 attacks Gobbers2
The attack failed
Gobbers2 attacks Gobbers1
The attack failed
Orc2 attacks Gobbers1
The attack failed
Round: 3
Gobbers1 attacks Gobbers2
The attack failed
Orc1 attacks Gobbers2
Attack hits for 6 damage!
Gobbers2 attacks Gobbers1
Attack hits for 5 damage!
Orc2 attacks Orc1
Attack hits for 9 damage!
Round: 4
Gobbers1 attacks Gobbers2
The attack failed
Orc1 is in RAAAAAGE
Orc1 attacks Gobbers2
The attack failed
Gobbers2 attacks Gobbers1
The attack failed
Orc2 is in RAAAAAGE
Orc2 attacks Gobbers1
The attack failed
Round: 5
Gobbers1 attacks Orc2
Attack hits for 4 damage!
Orc1 cooled down
Orc1 attacks Orc2
Attack hits for 9 damage!
Gobbers2 attacks Gobbers1
The attack failed
Orc2 cooled down
Orc2 attacks Orc1
Attack hits for 6 damag

This uses the playgame function again and pits the Goblin and Orc team against another Goblin and Orc.

In [5]:
class Warrior(Creature):
    def __init__(self, name, HP=50):
        Creature.__init__(self, name, HP)
        self.abilities["Attack"] = 5
        self.abilities["Defense"] = 10
        self.abilities["Speed"] = 4
        self.shield = False
    
    def shield_up(self):
        if self.shield == False:
            self.abilities["Attack"] -= 4
            self.abilities["Defense"] += 4
            self.shield = True
            print(f"{self.name} uses Shield Up!")
    
    def shield_down(self):
        if self.shield == True:
            self.abilities["Attack"] += 4
            self.abilities["Defense"] -= 4
            self.shield = False
            print(f"{self.name} uses shield down!")
    
    def turn(self, round_num, target_list):
        target = self.auto_select(target_list)
        if target == None:
            return False
        if round_num % 4 == 1:
            damage = self.attack(target)
            self.shield_up()
        elif round_num % 4 == 0:
            self.shield_down()
            damage = self.attack(target)
        else: 
            damage = self.attack(target)       
        if target.check_life() == 0:
            target_list.remove(target)
        return True

Warrior constructs the same as past classes with the added attribute of self.shield. 

shield_up() and shield_down() work the same as the Orcs heavy attack using a flag variable.

turn() - works the same as Orc but with different modulo requirements - it also doesn't use super.attack as there has been no other new defition of attack in Warrior. 

In [6]:
Pippin = Orc("ORC")
Bilbo = Warrior("Warrior")

creature_list1 = [Pippin]
creature_list2 = [Bilbo]


testgame(creature_list1, creature_list2)

Round: 1
ORC attacks Warrior
The attack failed
Warrior attacks ORC
Attack hits for 7 damage!
Warrior uses Shield Up!
Round: 2
ORC attacks Warrior
The attack failed
Warrior attacks ORC
The attack failed
Round: 3
ORC attacks Warrior
The attack failed
Warrior attacks ORC
The attack failed
Round: 4
ORC is in RAAAAAGE
ORC attacks Warrior
The attack failed
Warrior uses shield down!
Warrior attacks ORC
The attack failed
Round: 5
ORC cooled down
ORC attacks Warrior
The attack failed
Warrior attacks ORC
Attack hits for 8 damage!
Warrior uses Shield Up!
Round: 6
ORC attacks Warrior
Attack hits for 6 damage!
Warrior attacks ORC
Attack hits for 2 damage!
Round: 7
ORC attacks Warrior
The attack failed
Warrior attacks ORC
Attack hits for 3 damage!
Round: 8
ORC is in RAAAAAGE
ORC attacks Warrior
The attack failed
Warrior uses shield down!
Warrior attacks ORC
The attack failed
Round: 9
ORC cooled down
ORC attacks Warrior
The attack failed
Warrior attacks ORC
Attack hits for 7 damage!
Warrior uses Shie

In [7]:
class Archer(Creature):
    def __init__(self, name, HP=30):
        Creature.__init__(self, name, HP)
        self.abilities["Attack"] = 7
        self.abilities["Defense"] = 9
        self.abilities["Speed"] = 8
        self.alt = False

    
    def power_shot(self, other):
        other_att_spd = other.abilities["Defense"] + other.abilities["Speed"]
        chance = max([random.randint(1,20) for x in range(2)])
        if self.abilities["Speed"] > other.abilities["Speed"]:
            chance += (self.abilities["Speed"] - other.abilities["Speed"])
        if self.alt == False:
            self.alt = True
            self.abilities["Attack"] += 3
            self.abilities["Defense"] -= 3
            print(f"{self.name} attack rises!\n{self.name} defense lowers!")
        print(f"{self.name} shoots {other.name}")
        if chance > other_att_spd:
            otherHP = other.get_HP()
            damage = self.abilities["Attack"] + random.randint(1,8)
            otherHP -= damage
            other.set_HP(otherHP)
            if damage > 0:
                print(f"Power Shot hits for {damage} damage!")
        else:
            print("The Power Shot failed")
    
    def attack(self, other):
        if self.alt == True:
            self.alt = False
            self.abilities["Attack"] -= 3
            self.abilities["Defense"] += 3  
            print(f"{self.name} abilities return to normal!")
        super().attack(other)
    
    def auto_select(self, target_list):
        temp_target = None
        for creature in target_list:
            if temp_target == None or creature.get_HP() < temp_target.get_HP():
                temp_target = creature
        return temp_target
    
    def turn(self, round_num, target_list):
        target = self.auto_select(target_list)
        if target == None:
            return False
        if round_num % 4 == 1:
            self.attack(target)
        else: 
            self.power_shot(target)       
        if target.check_life() == 0:
            target_list.remove(target)
        return True


class Fighter(Creature):
    def __init__(self, name, HP=50):
        Creature.__init__(self, name, HP)
        self.abilities["Attack"] = 5
        self.abilities["Defense"] = 8
        self.abilities["Speed"] = 5
    
    def auto_select(self, target_list):
        temp_target = None
        for creature in target_list:
            if temp_target == None or creature.get_HP() > temp_target.get_HP():
                temp_target = creature
        return temp_target
    
    def turn(self, round_num, target_list):
        target = self.auto_select(target_list)
        if target == None:
            return False
        self.attack(target)
        if target.check_life() == 0:
            target_list.remove(target)
            target = self.auto_select(target_list)
        if target == None:
            return False
        self.abilities["Attack"] -=3
        print(f"{self.name} unleashes a flurry of strikes!")
        for _ in range(2):
            if target == None:
                self.abilities["Attack"] +=3
                return False
            self.attack(target)
            if target.check_life() == 0:
                target_list.remove(target)
                target = self.auto_select(target_list)
        self.abilities["Attack"] +=3
        return True


The archer is created in the same style as the creature, calling the init for it and changing the variables where needed. It adds the attribute alt. 
Power_shot method - this method gets the enemies defense and speed, it then uses a list comprehension to get the max of two random integers. If the archers speed is greater than the enemies speed, the difference is added to the attack roll. self.alt is then checked to decide whether to power up or not, and adds the attack and subtracts the defense if needed. The archer then attacks if the if statement is True and if not, prints that it failed.
The normal attack is the same as for the Orcs attack.
The auto_select selects the target with the least HP using an if statement and variable.
The turn is similar to the previous turn structures.

The fighter is similar to the archer. It's autoselect is the same except that it goes for the highest hitpoints.
The turn is slightly different. It gets its' target and attacks it. It then checks if the target survived, and changes target if it didnt and ends if it couldnt select a target. The attack is then reduced and two more attacks are done at the lower attack level. If there is no more targets mid attack stream, the attack is raised and the function returns False, otherwise it raises the attack and returns True.


In [8]:
Pippin = Archer("Legolas")

Bilbo = Fighter("Aragorn")

creature_list1 = [Bilbo]
creature_list2 = [Pippin]


testgame(creature_list1, creature_list2)

Round: 1
Aragorn attacks Legolas
Attack hits for 9 damage!
Aragorn unleashes a flurry of strikes!
Aragorn attacks Legolas
The attack failed
Aragorn attacks Legolas
The attack failed
Legolas attacks Aragorn
Attack hits for 11 damage!
Round: 2
Aragorn attacks Legolas
The attack failed
Aragorn unleashes a flurry of strikes!
Aragorn attacks Legolas
The attack failed
Aragorn attacks Legolas
The attack failed
Legolas attack rises!
Legolas defense lowers!
Legolas shoots Aragorn
Power Shot hits for 14 damage!
Round: 3
Aragorn attacks Legolas
Attack hits for 6 damage!
Aragorn unleashes a flurry of strikes!
Aragorn attacks Legolas
The attack failed
Aragorn attacks Legolas
Attack hits for 6 damage!
Legolas shoots Aragorn
Power Shot hits for 16 damage!
Round: 4
Aragorn attacks Legolas
The attack failed
Aragorn unleashes a flurry of strikes!
Aragorn attacks Legolas
The attack failed
Aragorn attacks Legolas
Attack hits for 5 damage!
Legolas shoots Aragorn
Power Shot hits for 16 damage!
Aragorn has b

In [9]:
class OrcGeneral(Orc, Warrior):
    def __init__(self, name, HP=80):
        Orc.__init__(self, name, HP)
        self.shield = False
    
    def turn(self, round_num, target_list):
        target = self.auto_select(target_list)
        if target == None:
            return False
        if round_num % 4 == 1:
            self.attack(target)
            self.shield_up()
        elif round_num % 4 == 2: 
            self.attack(target) 
        elif round_num % 4 == 3:
            self.shield_down()
            self.attack(target)
        else:
            self.heavy_attack(target)      
        if target.check_life() == 0:
            target_list.remove(target)
        return True
    

class GoblinKing(Archer, Goblin):
    def __init__(self, name, HP=50):
        Goblin.__init__(self, name, HP)
        self.alt = False

   
class Boss(Orc):
    def __init__(self, name, HP=200):
        Orc.__init__(self, name, HP)
        self.abilities["Attack"] = 5
        self.abilities["Defense"] = 8
        self.abilities["Speed"] = 5

    def auto_select(self, target_list, mode):
        temp_target = None
        if mode == "Random":
            mode = random.choice(["Strong", "Weak"])
        if mode == "Strong":
            for creature in target_list:
                if temp_target == None or creature.get_HP() > temp_target.get_HP():
                    temp_target = creature
        elif mode == "Weak":
            for creature in target_list:
                if temp_target == None or creature.get_HP() < temp_target.get_HP():
                    temp_target = creature
        return temp_target

    
    def turn(self, round_num, target_list):
        if round_num % 4 == 1:
            target = self.auto_select(target_list, "Weak")
            if target == None:
                return False
            self.attack(target)
            if target.check_life() == 0:
                target_list.remove(target)
                target = self.auto_select(target_list, "Random")
            self.abilities["Attack"] -=3
            print(f"{self.name} unleashes a flurry of strikes!")
            for _ in range(2):
                if target == None:
                    self.abilities["Attack"] += 3
                    return False
                self.attack(target)
                if target.check_life() == 0:
                    target_list.remove(target)
                    target = self.auto_select(target_list, "Random")
            self.abilities["Attack"] +=3
            return True
        else:
            target = self.auto_select(target_list, "Strong")
            if target == None:
                return False
            self.heavy_attack(target)
            if target.check_life() == 0:
                target_list.remove(target)
            return True


    
The OrcGeneral inherits from both the Orc and Warrior. It's mro is OrcGeneral, Orc, Warrior, Creature, Object.
The constructor uses the Orc constructor and then adds the self.shield attribute so that its' warrior shields work
The turn method is similar to other methods for turn using modulo.

The Goblin King uses the Goblin constructor and then the Archers methods. 

The Boss class inherits from the Orc. Its autoselect checks the input mode and then attacks based on that mode. For random it uses the random module to choose its attack.
The turn method for the boss is a mix of the fighters and an Orc attack. It uses the same modulo if statements as before. For teh target selection it follows the outlined rules.

In [10]:
OrcGen = OrcGeneral("OrcGen")
GoblinKinger = GoblinKing("GoblinKing")

creature_list1 = [OrcGen]
creature_list2 = [GoblinKinger]

testgame(creature_list1, creature_list2)

Round: 1
OrcGen attacks GoblinKing
Attack hits for 6 damage!
OrcGen uses Shield Up!
GoblinKing attacks OrcGen
The attack failed
Round: 2
OrcGen attacks GoblinKing
The attack failed
GoblinKing attack rises!
GoblinKing defense lowers!
GoblinKing shoots OrcGen
The Power Shot failed
Round: 3
OrcGen uses shield down!
OrcGen attacks GoblinKing
The attack failed
GoblinKing shoots OrcGen
Power Shot hits for 9 damage!
Round: 4
OrcGen is in RAAAAAGE
OrcGen attacks GoblinKing
The attack failed
GoblinKing shoots OrcGen
Power Shot hits for 10 damage!
Round: 5
OrcGen cooled down
OrcGen attacks GoblinKing
The attack failed
OrcGen uses Shield Up!
GoblinKing abilities return to normal!
GoblinKing attacks OrcGen
The attack failed
Round: 6
OrcGen attacks GoblinKing
The attack failed
GoblinKing attack rises!
GoblinKing defense lowers!
GoblinKing shoots OrcGen
Power Shot hits for 7 damage!
Round: 7
OrcGen uses shield down!
OrcGen attacks GoblinKing
Attack hits for 8 damage!
GoblinKing shoots OrcGen
Power S

In [11]:
class Wizard(Creature):
    def __init__(self, name, HP= 20):
        Creature.__init__(self, name, HP)
        self.abilities["Attack"] = 3
        self.abilities["Defense"] = 5
        self.abilities["Speed"] = 5
        self.abilities["Arcana"] = 10
        self.mana = 100

    def select_target(self, target_list):
        if len(target_list) == 0:
            return None
        if len(target_list) == 1:
            print("\033[93m" +"Target automatically chosen as there is only one option!"+ "\033[0m")
            return target_list[0]
        print("Select target:")
        for count, target in enumerate(target_list, 1):
            print(f"{count}: {target.name}, {target.get_HP()}/{target.maxHP}")
        choose = int(input("Chose the number of the target you want to attack: "))-1
        while not 0 <= choose <= len(target_list)-1:
            choose = int(input("Chose the number of the target you want to attack: "))-1            
        chosen_target = target_list[choose]
        return chosen_target
 
    def add_mana(self, amount):
        if self.mana == 100:
            print("\033[94m" + "Mana is full!" + "\033[0m")
        else:
            self.mana += amount
            if self.mana > 100:
                self.mana = 100
            print("Mana:" +"\033[94m" + f" +{amount}!" + "\033[0m")

    def attack(self, target):
        super().attack(target)
        self.add_mana(20)

    def recharge(self):
        self.add_mana(30)

    def fire_bolt(self, target):
        print(f"{self.name} fires a fire bolt at {target.name}")
        other_att_spd = target.abilities["Defense"] + target.abilities["Speed"]
        chance = random.randint(1,20) + self.abilities["Arcana"]//2
        if chance > other_att_spd:
            otherHP = target.get_HP()
            damage = random.randint(1,self.abilities["Arcana"])
            otherHP -= damage
            target.set_HP(otherHP)
            if damage > 0:
                print(f"Fire bolt hits for {damage} fire damage!")
            self.add_mana(10)
        else:
            print("The attack failed")
    
    def fire_storm(self, enemies):
        if self.mana >= 50:
            self.mana -= 50
            print("Mana:" +"\033[94m" + "-50" + "\033[0m")
            for enemy in enemies:
                attack = random.randint(1,20) + self.abilities["Speed"]
                damage = random.randint(5,20) + self.abilities["Arcana"]
                if attack >= self.abilities["Arcana"]:
                    damage = damage // 2
                enemy.set_HP((enemy.get_HP()-damage))
                print(f"Fire storm deals {enemy.name} for {damage} fire damage!") 
                if enemy.check_life() == 0:
                    enemies.remove(enemy)
        else:
            print("Not enough mana")
    
    def heal(self, other):
        if self.mana >= 20:
            self.mana -= 20
            print("Mana:" +"\033[94m" + "-20" + "\033[0m")
            amount_to_heal = random.randint(0,8) + self.abilities["Arcana"]//2
            other.set_HP((other.get_HP() + amount_to_heal))
            print(f"{self.name} heals {other.name} for {amount_to_heal} HP!")
        else:
            print("Not enough mana")

    def mass_heal(self, allies):
        if self.mana >= 30:
            self.mana -= 30
            print("Mana:" +"\033[94m" + "-30" + "\033[0m")
            for ally in allies:
                amount_to_heal = random.randint(0,10) + self.abilities["Arcana"]
                ally.set_HP((ally.get_HP() + amount_to_heal))
                print(f"{self.name} heals {ally.name} for {amount_to_heal} HP!")
        else:
            print("Not enough mana")

    def player_turn(self, alliesList, enemiesList):
        def display_info(character):
            print(f"{character.name} HP:{character.get_HP()}/{character.maxHP} ")
        if len(enemiesList) == 0 or len(alliesList)==0:
            return False
        print("=======================================================")
        print(f"Player: {self.name} HP:{self.get_HP()}/{self.maxHP} Mana: " +"\033[94m" + f"{self.mana}/100"+ "\033[0m")
        print("Allies:")
        for char in alliesList:
            if char.__class__.__name__ == "Wizard":
                continue
            display_info(char)
        print("Enemies:")
        for char in enemiesList:
            display_info(char)
        print("=======================================================")
        print("""
              Actions. F: Attack R: Recharge Mana
              Spells. 1: Heal 2: Firebolt 3: Mass Heal 4: Fire Storm
              To Quit game type: Quit
              """)
        print("=======================================================")
        action = input("Enter action: ").upper()
        while action not in ["1","2","3","4","R","F"] and action != "QUIT":
            action = input("Enter action: ").upper()
        if action == "QUIT":
            return "Quit"
        if action == "4":
            self.fire_storm(enemiesList)
        elif action in "F2":
            target = self.select_target(enemiesList)
            if action in "F":
                self.attack(target)
            elif action == "2":
                self.fire_bolt(target)
            if target.check_life() == 0:
                enemiesList.remove(target)
        elif action == "3":
            self.mass_heal(alliesList)
        elif action == "1":
            target = self.select_target(alliesList)  
            self.heal(target)
        elif action in "R":
            self.recharge()
        return True


The wizard inits from the creature class. It has the attribute mana for tracking its mana usage.
For select_target it first checks that there are targets to select. If the target list is only of length 1 it prints a messsage and automatically chooses that target. If not, it will print out the target list and numbers for the target to be chosen by.

The add mana will print out messages to let the user know the mana level. These are set to the color blue to be more visible. I added an add_mana method to centralize adding mana and reduce redundant code. If the adding of mana puts its over 100 it is set to 100. Either way it prints the amount that should have been added.

attack calls the super().attack method and then uses the add mana method to add the required mana.

Recharge also uses the add_mana method to add the required mana

fire_bolt works like a normal attack but adds the floor division of arcana to the attack chance. It also rolls 
for damage between 1 and the arcana amount and adds mana back after the attack.

fire_storm checks that the mana is 50 or greater before executing. It then subtracts the mana and get an attack variable between 1-20 plus speed. It gets a damage variable between 5-20 and adds arcana. If the attack is greater than or equal to arcana the attack is floor divided by 2.

heal - this will check if mana is greater than or equal to 20 and subtract if it is before healing for a random amount between 0-8 plus floor divided arcana by 2.
mass heal - this will work the same as heal but for an amount between 0-10 plus arcana

player_turn first checks that there is either enemies or allies. If there aren't it returns False. It then prints out the status of each ally, checking if it's a wizard so it skips the user, and enemy. It then checks the user input and returns the action or calls the method.

In [12]:
wiz = Wizard("Gandalf")

OrcGen = OrcGeneral("OrcGen")
BigBoss = Boss("BigBoss")

creature_list = [OrcGen,  BigBoss]

target = creature_list[0]
wiz.fire_bolt(target)
wiz.fire_storm(creature_list)
print(target.HP)
print(target.maxHP)
wiz.heal(target)
wiz.recharge()
print("mana:",wiz.mana)
print(target.HP)
wiz.mass_heal(creature_list)
print(target.HP)


Gandalf fires a fire bolt at OrcGen
Fire bolt hits for 9 fire damage!
Mana is full!
Mana:-50
Fire storm deals OrcGen for 12 fire damage!
Fire storm deals BigBoss for 25 fire damage!
59
80
Mana:-20
Gandalf heals OrcGen for 6 HP!
Mana: +30!
mana: 60
65
Mana:-30
Gandalf heals OrcGen for 10 HP!
Gandalf heals BigBoss for 10 HP!
75


The above shows the Wizard objects methods being used.

In [13]:
class Battle():
    def __init__(self):
        self.bossCheck = False
        self.Goblinking = GoblinKing("GobblerKing")
        self.OrcGen = OrcGeneral("GeneralOrca")
        self.Minigobbler = Goblin("MiniGobbler")
        self.Orca = Orc("Orca") 
        self.enemies = [self.Goblinking, self.OrcGen, self.Minigobbler, self.Orca]

        self.Fighty = Fighter("Fighty")
        self.Archy = Archer("Archy")
        self.Wario = Warrior("Wario")
        self.Cretin = Creature("Cretin")
        self.allies = [self.Fighty, self.Archy, self.Wario, self.Cretin]

        self.BigBoss = Boss("BigBoss")

        self.GWiz = Wizard("G-Wiz")
    
    def start(self):
        self.all_players = []
        for char in self.allies:
            self.all_players.append(char)
        for char in self.enemies:
            self.all_players.append(char)
        play = input("Do you want to play the Wizard?: Yes / No").lower()
        while play not in ["yes", "no"]:
            play = input("Do you want to play the Wizard?: Yes / No").lower()
        if play == "yes":
            self.all_players.append(self.GWiz)
            self.allies.append(self.GWiz)
        self.all_players.sort(key= lambda x: x.abilities["Speed"],reverse=True )

        print("THE BATTLE BEGINS")

        doTurn = True
        playerQuit = False

        for round in range(1,1000):
            if not doTurn:
                break
            print("=========================")
            print(f"Round: {round}")
            print("=========================")


            for creature in self.all_players:
                if creature.get_HP() == 0:
                    continue
                if doTurn:
                    if creature.__class__.__name__ == "Wizard":
                        doTurn = creature.player_turn(self.allies, self.enemies)
                        if doTurn == "Quit":
                            print("Player quit early!")
                            playerQuit = True
                            break
                    elif creature in self.allies:
                        doTurn = creature.turn(round, self.enemies)
                    elif creature in self.enemies:
                        doTurn = creature.turn(round, self.allies)
                if self.GWiz.get_HP() == 0:
                    break
                if len(self.allies) == 0 or len(self.enemies) == 0:
                    break
                time.sleep(.5)
            
            if playerQuit == True:
                break
            print("=========================")
            print(f"End of round {round}")

                    
            if len(self.enemies) == 0 and self.bossCheck == False:
                self.enemies.append(self.BigBoss)
                self.all_players.append(self.BigBoss)
                self.bossCheck = True
                doTurn = True
            if len(self.allies) != 0 or len(self.enemies) != 0:
                doTurn = True
            if len(self.allies) == 0 or len(self.enemies) == 0 or self.GWiz.get_HP() == 0:
                doTurn = False
        
        if len(self.allies) == 0 or self.GWiz.get_HP() == 0:
            print("=========================")
            print("The Evils won!!")
        if len(self.enemies) == 0:
            print("=========================")
            print("The Friendlies won!!")



Battle class is used to run the battle.
The constructor starts a boss check which will be used to check if the boss has been sent out.
It then starts an instance of each ally and enemy that is desired. These are added to lists of self.enemies and self.allies.
self.Bigboss and self.GWiz are not added yet to either list but are initialised.


The start method puts all the characters into a main list, self.all_players. It then checks if the user wants to play the wizard.
If the user wants to play the wizard, its added to allies and all players, if not it is skipped and the game plays without it.
The list is sorted using the abilities["Speed"] in reverse for quickest to slowest.

The range is set to 1000 so that the program will have time to finish but won't run forever.
If doTurn is set to False it signals that one of the requisites has been met and to go to the end to see if it's game over.
PlayerQuit is used purely if the player quits early.

Each round it loops through the creatures in all players. Where HP is not equal to 0 the creature does a turn.
It first checks if the creature is the wizard and calls its unique turn method. If the players quits it sets playerQuit to False and breaks the loop
It will also check if self.GWiz has healthh and if the allies list or enemies list is empty. 
I added time.sleep(.5) to make it more readable as it scrolls, but not too much that the round takes ages.

Finally the checks for winning are done. If the enemies is 0 and bossCheck is False it will add the boss in and continue the game.
Otherwise it will end the game with whichever team won, based on the if statement requirements.

In [14]:
# Starts an object of Battle and runs the method to start the battle.
battle = Battle()
battle.start()


THE BATTLE BEGINS
Round: 1
Archy attacks MiniGobbler
The attack failed
GobblerKing attacks Cretin
Attack hits for 7 damage!
MiniGobbler attacks Fighty
Attack hits for 5 damage!
Fighty attacks GeneralOrca
The attack failed
Fighty unleashes a flurry of strikes!
Fighty attacks GeneralOrca
The attack failed
Fighty attacks GeneralOrca
Attack hits for 6 damage!
Cretin attacks Orca
Attack hits for 4 damage!
Wario attacks GobblerKing
The attack failed
Wario uses Shield Up!
GeneralOrca attacks Archy
The attack failed
GeneralOrca uses Shield Up!
Orca attacks Cretin
Attack hits for 6 damage!
Cretin has been dusted.
End of round 1
Round: 2
Archy attack rises!
Archy defense lowers!
Archy shoots MiniGobbler
Power Shot hits for 13 damage!
GobblerKing attack rises!
GobblerKing defense lowers!
GobblerKing shoots Archy
Power Shot hits for 8 damage!
MiniGobbler attacks Archy
The attack failed
Fighty attacks GeneralOrca
The attack failed
Fighty unleashes a flurry of strikes!
Fighty attacks GeneralOrca
Att